## Data wrangling

In [1]:
import pandas as pd
import numpy as np
import glob
import preprocessing as pre
import seaborn as sn
import matplotlib.pyplot as plt

%matplotlib inline

First we need to extract data from the excel file. In order to construct the eventual for loop, we'll first need to try extracting data from one file

In [2]:
pwd

'/home/bea/bhs2020/Projet_BHS2020/SNF_ADHDsubtypes_project'

In [3]:
df_eeg = pre.process_all_excel_files()
df_eeg

,electrode,brain_oscillation,fft_abs_power,freq_band,id
0,FP1,Delta,15.565495,1.0-4.0Hz,134
1,FP2,Delta,13.945462,1.0-4.0Hz,134
2,F7,Delta,10.874886,1.0-4.0Hz,134
3,F3,Delta,13.581803,1.0-4.0Hz,134
4,Fz,Delta,15.644595,1.0-4.0Hz,134
...,...,...,...,...,...
7671,Pz,Beta,4.780903,12.0-25.0Hz,87
7672,P4,Beta,4.276557,12.0-25.0Hz,87
7673,T6,Beta,3.603100,12.0-25.0Hz,87
7674,O1,Beta,12.159364,12.0-25.0Hz,87


Reorder columns of df

In [4]:
df_eeg = df_eeg[['id', 'brain_oscillation','freq_band','electrode','fft_abs_power']]

Remove participants (10,18, 52, 215 and 617) because of missing Neuropsy data

In [5]:
df_eeg = df_eeg[(df_eeg.id != '10') & (df_eeg.id != '18') & (df_eeg.id != '52') & (df_eeg.id != '215') & (df_eeg.id != '617')]

Verify data wrangling

In [6]:
print(df_eeg.shape)
df_eeg.head(5)

(7296, 5)


,id,brain_oscillation,freq_band,electrode,fft_abs_power
0,134,Delta,1.0-4.0Hz,FP1,15.565495
1,134,Delta,1.0-4.0Hz,FP2,13.945462
2,134,Delta,1.0-4.0Hz,F7,10.874886
3,134,Delta,1.0-4.0Hz,F3,13.581803
4,134,Delta,1.0-4.0Hz,Fz,15.644595


In [7]:
#sort by ids
df_eeg = df_eeg.sort_values(by=['id'])

In [8]:
df_eeg.dtypes

id                   object
brain_oscillation    object
freq_band            object
electrode            object
fft_abs_power        object
dtype: object

In [9]:
print('# unique ids: {}'.format(len(df_eeg.id.unique())))
print(df_eeg.id.value_counts())

# unique ids: 96
21     76
41     76
400    76
03     76
213    76
       ..
212    76
411    76
89     76
84     76
16     76
Name: id, Length: 96, dtype: int64


In [10]:
print(df_eeg.electrode.value_counts())
print(df_eeg.brain_oscillation.value_counts())
print(df_eeg.freq_band.value_counts())

O2     384
F7     384
C4     384
T6     384
Fz     384
P3     384
F4     384
FP2    384
Cz     384
F3     384
FP1    384
O1     384
F8     384
T4     384
C3     384
Pz     384
P4     384
T5     384
T3     384
Name: electrode, dtype: int64
Alpha    1824
Theta    1824
Beta     1824
Delta    1824
Name: brain_oscillation, dtype: int64
1.0-4.0Hz      1824
8.0-12.0Hz     1824
4.0-8.0Hz      1824
12.0-25.0Hz    1824
Name: freq_band, dtype: int64


### Merge dataframes (Neuropsy data with df (eeg))

#### We now need to import the Neuropsydata

In [11]:
df_neuropsy = pd.read_excel("Neuropsy.xlsx", na_values=".")
print(df_neuropsy.shape)
df_neuropsy.head(5)

(100, 15)


,ID,Age,Gender,cIM,cHR,cIE,cSC,inat,hyper,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
1,3,20,1,10.0,5.0,13.0,1.0,8.0,5.0,27.0,34.0,41.0,25.0,31.0,38.0
2,4,18,1,26.0,17.0,7.0,15.0,23.0,11.0,93.0,89.0,96.0,90.0,92.0,90.0
3,7,23,1,24.0,8.0,6.0,14.0,19.0,3.0,86.0,66.0,112.0,94.0,90.0,100.0
4,10,18,1,NaN,NaN,NaN,NaN,NaN,NaN,98.0,103.0,93.0,92.0,100.0,85.0


Then remove participants (10,18, 52 and 215) because of missing Neuropsy data

In [12]:
df_neuropsy= df_neuropsy.dropna(axis=0, how='any')

Verify it worked

In [13]:
print(df_neuropsy.shape)

(96, 15)


Rename ID variable

In [14]:
#rename id
df_neuropsy.rename(columns = {'ID':'id'}, inplace = True) #rename id so it can be merged
df_neuropsy.head(5)

,id,Age,Gender,cIM,cHR,cIE,cSC,inat,hyper,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
1,3,20,1,10.0,5.0,13.0,1.0,8.0,5.0,27.0,34.0,41.0,25.0,31.0,38.0
2,4,18,1,26.0,17.0,7.0,15.0,23.0,11.0,93.0,89.0,96.0,90.0,92.0,90.0
3,7,23,1,24.0,8.0,6.0,14.0,19.0,3.0,86.0,66.0,112.0,94.0,90.0,100.0
5,11,21,1,16.0,26.0,13.0,10.0,17.0,13.0,45.0,33.0,78.0,69.0,51.0,94.0


#### Now let's merge the two dataframes together

In [15]:
#change dtype of id column from df
df_eeg['id']=df_eeg['id'].astype(int)

In [16]:
df_full = pd.merge(left=df_eeg, right=df_neuropsy, left_on='id', right_on='id')

In [17]:
df_full.head()

,id,brain_oscillation,freq_band,electrode,fft_abs_power,Age,Gender,cIM,cHR,cIE,cSC,inat,hyper,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis
0,1,Delta,1.0-4.0Hz,FP1,15.376173,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
1,1,Theta,4.0-8.0Hz,F3,3.845777,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
2,1,Theta,4.0-8.0Hz,Fz,4.259641,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
3,1,Theta,4.0-8.0Hz,F4,3.552830,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0
4,1,Theta,4.0-8.0Hz,F8,2.064215,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0


In [18]:
#sort by ids
df_full = df_full.sort_values(by=['id'])

### Subtype labelling according to clinical standard

Create a column for categorization of ADHD subtypes

In [19]:
df_full['subtype'] = df_full.apply(lambda x: pre.categorize_subtypes(x['inat'], x['hyper']), axis=1)

df_full.head(5)

,id,brain_oscillation,freq_band,electrode,fft_abs_power,Age,Gender,cIM,cHR,cIE,cSC,inat,hyper,Aqtot,Aqaudi,Aqvis,RCQtot,RCQaudi,RCQvis,subtype
0,1,Delta,1.0-4.0Hz,FP1,15.376173,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0,mixed
54,1,Delta,1.0-4.0Hz,F7,7.968847,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0,mixed
53,1,Beta,12.0-25.0Hz,FP2,13.148877,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0,mixed
52,1,Beta,12.0-25.0Hz,T6,4.664137,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0,mixed
51,1,Beta,12.0-25.0Hz,P4,6.975211,21,1,17.0,31.0,29.0,9.0,18.0,20.0,90.0,91.0,92.0,94.0,80.0,110.0,mixed


In [20]:
#encoding the subtypes into numerocal labels for future SNF analysis/machine learning
# hyperactive = 0
# inattentive subtype = 1
# mixed subtype = 2
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df_full["adhdtype"] = lb_make.fit_transform(df_full["subtype"])
df_full[["subtype", "adhdtype"]].head(11)


,subtype,adhdtype
0,mixed,2
54,mixed,2
53,mixed,2
52,mixed,2
51,mixed,2
50,mixed,2
49,mixed,2
48,mixed,2
47,mixed,2
46,mixed,2


In [21]:
#Reorder columns 
df_full = df_full[['id','Age', 'Gender','inat','hyper',"subtype",'adhdtype', 'brain_oscillation','electrode','fft_abs_power', 'cIM', 'cHR', 'cIE', 'cSC', 'Aqtot', 'Aqaudi', 'Aqvis', 'RCQtot', 'RCQaudi', 'RCQvis']]
df_full.head()
df_full['adhdtype'].value_counts()

1    3648
2    3496
0     152
Name: adhdtype, dtype: int64

Adjust data types in dataframe

In [22]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7296 entries, 0 to 5851
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7296 non-null   int64  
 1   Age                7296 non-null   int64  
 2   Gender             7296 non-null   int64  
 3   inat               7296 non-null   float64
 4   hyper              7296 non-null   float64
 5   subtype            7296 non-null   object 
 6   adhdtype           7296 non-null   int64  
 7   brain_oscillation  7296 non-null   object 
 8   electrode          7296 non-null   object 
 9   fft_abs_power      7296 non-null   object 
 10  cIM                7296 non-null   float64
 11  cHR                7296 non-null   float64
 12  cIE                7296 non-null   float64
 13  cSC                7296 non-null   float64
 14  Aqtot              7296 non-null   float64
 15  Aqaudi             7296 non-null   float64
 16  Aqvis              7296 

In [23]:
df_full['fft_abs_power'] = df_full['fft_abs_power'].astype(float)
print(df_full.dtypes)

id                     int64
Age                    int64
Gender                 int64
inat                 float64
hyper                float64
subtype               object
adhdtype               int64
brain_oscillation     object
electrode             object
fft_abs_power        float64
cIM                  float64
cHR                  float64
cIE                  float64
cSC                  float64
Aqtot                float64
Aqaudi               float64
Aqvis                float64
RCQtot               float64
RCQaudi              float64
RCQvis               float64
dtype: object


In [24]:
df_full_eeg = df_full[['id', 'Gender', 'adhdtype','brain_oscillation','electrode', 'fft_abs_power']]
df_full_eeg

,id,Gender,adhdtype,brain_oscillation,electrode,fft_abs_power
0,1,1,2,Delta,FP1,15.376173
54,1,1,2,Delta,F7,7.968847
53,1,1,2,Beta,FP2,13.148877
52,1,1,2,Beta,T6,4.664137
51,1,1,2,Beta,P4,6.975211
...,...,...,...,...,...,...
5798,702,1,1,Delta,Fz,20.934284
5797,702,1,1,Delta,F3,17.471239
5796,702,1,1,Delta,F7,11.253618
5803,702,1,1,Delta,Cz,23.281091


### Descriptive statistics according to gender (male and female)

Clean dataset and keep variables of interest only

In [31]:
df_analysis = df_full[['id','Gender','subtype','adhdtype','brain_oscillation', 'fft_abs_power', 'cIM', 'cHR', 'cIE', 'cSC', 'Aqtot', 'Aqaudi', 'Aqvis', 'RCQtot', 'RCQaudi', 'RCQvis']]
print(df_analysis.head())

    id  Gender subtype  adhdtype brain_oscillation  fft_abs_power   cIM   cHR  \
0    1       1   mixed         2             Delta      15.376173  17.0  31.0   
54   1       1   mixed         2             Delta       7.968847  17.0  31.0   
53   1       1   mixed         2              Beta      13.148877  17.0  31.0   
52   1       1   mixed         2              Beta       4.664137  17.0  31.0   
51   1       1   mixed         2              Beta       6.975211  17.0  31.0   

     cIE  cSC  Aqtot  Aqaudi  Aqvis  RCQtot  RCQaudi  RCQvis  
0   29.0  9.0   90.0    91.0   92.0    94.0     80.0   110.0  
54  29.0  9.0   90.0    91.0   92.0    94.0     80.0   110.0  
53  29.0  9.0   90.0    91.0   92.0    94.0     80.0   110.0  
52  29.0  9.0   90.0    91.0   92.0    94.0     80.0   110.0  
51  29.0  9.0   90.0    91.0   92.0    94.0     80.0   110.0  


### Description statistics by gender, subtype and brain oscillations

In [32]:
df_analysis.groupby(['Gender','subtype', 'brain_oscillation']).describe()

id                                      \
                                  count        mean         std   min   25%   
Gender subtype brain_oscillation                                              
1      inat    Alpha              646.0  179.764706  185.621020   4.0  64.0   
               Beta               646.0  179.764706  185.621020   4.0  64.0   
               Delta              646.0  179.764706  185.621020   4.0  64.0   
               Theta              646.0  179.764706  185.621020   4.0  64.0   
       mixed   Alpha              437.0  154.869565  189.447661   1.0  14.0   
               Beta               437.0  154.869565  189.447661   1.0  14.0   
               Delta              437.0  154.869565  189.447661   1.0  14.0   
               Theta              437.0  154.869565  189.447661   1.0  14.0   
2      hyper   Alpha               38.0   77.500000   18.748333  59.0  59.0   
               Beta                38.0   77.500000   18.748333  59.0  59.0   
               Delta               38.0   77.500000   18.748333  59.0  59.0   
               Theta               38.0   77.500000   18.748333  59.0  59.0   
       inat    Alpha              266.0  108.000000  126.600486  19.0  24.0   
               Beta               266.0  108.000000  126.600486  19.0  24.0   
               Delta              266.0  108.000000  126.600486  19.0  24.0   
               Theta              266.0  108.000000  126.600486  19.0  24.0   
       mixed   Alpha              437.0  179.695652  174.422287  17.0  45.0   
               Beta               437.0  179.695652  174.422287  17.0  45.0   
               Delta              437.0  179.695652  174.422287  17.0  45.0   
               Theta              437.0  179.695652  174.422287  17.0  45.0   

                                                      adhdtype       ...  \
                                    50%    75%    max    count mean  ...   
Gender subtype brain_oscillation                                     ...   
1      inat    Alpha               84.0  220.0  702.0    646.0  1.0  ...   
               Beta                84.0  220.0  702.0    646.0  1.0  ...   
               Delta               84.0  220.0  702.0    646.0  1.0  ...   
               Theta               84.0  220.0  702.0    646.0  1.0  ...   
       mixed   Alpha               80.0  213.0  701.0    437.0  2.0  ...   
               Beta                80.0  213.0  701.0    437.0  2.0  ...   
               Delta               80.0  213.0  701.0    437.0  2.0  ...   
               Theta               80.0  213.0  701.0    437.0  2.0  ...   
2      hyper   Alpha               77.5   96.0   96.0     38.0  0.0  ...   
               Beta                77.5   96.0   96.0     38.0  0.0  ...   
               Delta               77.5   96.0   96.0     38.0  0.0  ...   
               Theta               77.5   96.0   96.0     38.0  0.0  ...   
       inat    Alpha               61.5  121.0  409.0    266.0  1.0  ...   
               Beta                61.5  121.0  409.0    266.0  1.0  ...   
               Delta               61.5  121.0  409.0    266.0  1.0  ...   
               Theta               61.5  121.0  409.0    266.0  1.0  ...   
       mixed   Alpha              126.0  216.0  700.0    437.0  2.0  ...   
               Beta               126.0  216.0  700.0    437.0  2.0  ...   
               Delta              126.0  216.0  700.0    437.0  2.0  ...   
               Theta              126.0  216.0  700.0    437.0  2.0  ...   

                                 RCQaudi        RCQvis                        \
                                     75%    max  count       mean        std   
Gender subtype brain_oscillation                                               
1      inat    Alpha                94.0  113.0  646.0  76.558824  24.811515   
               Beta                 94.0  113.0  646.0  76.558824  24.811515   
               Delta                94.0  113.0  646.0  76.558824  24.

#### Create two different df for behavioral data and connors

In [37]:
df_connors = df_analysis[['id','Gender','cIM', 'cHR', 'cIE', 'cSC']]
df_behavioral = df_analysis[['id','Gender', 'Aqtot', 'Aqaudi', 'Aqvis', 'RCQtot', 'RCQaudi', 'RCQvis']]
df_labels = df_analysis[['id','adhdtype']]

df_labels

,id,adhdtype
0,1,2
54,1,2
53,1,2
52,1,2
51,1,2
...,...,...
5798,702,1
5797,702,1
5796,702,1
5803,702,1


#### Export as csv 

In [34]:
df_connors.to_csv('Data/df_connors')
df_behavioral.to_csv('Data/df_behavioral')
df_labels.to_csv('Data/labels')

#### Export df_eeg into csv format for SNF analysis

In [35]:
df_full_eeg.to_csv('Data/df_eeg')

#### Export df_analysis into csv format for visualization

In [36]:
df_analysis.to_csv('Data/df_analysis')